<a href="https://colab.research.google.com/github/kevinscaria/InstructABSA/blob/main/JointTask_Training_%26_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Libraries

In [ ]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [ ]:
if IN_COLAB:
  !pip install transformers
  !pip install datasets
  !pip install evaluate
  !pip install sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 17.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which 

In [ ]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"
os.environ["TRANSFORMERS_NO_TF"] = "1"
os.environ["WANDB_DISABLED"] = "true"
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"
os.environ["TF_USE_LEGACY_KERAS"] = "1"
os.environ["HF_HUB_DISABLE_XET"] = "1"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import torch

if IN_COLAB:
    root_path = '/content/drive/My Drive/instructABSA'
else:
    root_path = 'Enter local path'

use_mps = True if torch.backends.mps.is_built() else False
os.chdir(root_path)

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd

from InstructABSA.data_prep import DatasetLoader
from InstructABSA.utils import T5Generator, T5Classifier
from instructions import InstructionsHandler

## Training

In [ ]:
task_name = 'joint_task'
experiment_name = 'aspe-absa2'
model_checkpoint = 'kevinscaria/joint_tk-instruct-base-def-pos-neg-neut-combined'
print('Experiment Name: ', experiment_name)
model_out_path = './Models'
model_out_path = os.path.join(model_out_path, task_name, f"{model_checkpoint.replace('/', '')}-{experiment_name}")
print('Model output path: ', model_out_path)

Experiment Name:  aspe-absa2
Model output path:  ./Models/joint_task/kevinscariajoint_tk-instruct-base-def-pos-neg-neut-combined-aspe-absa2


In [ ]:
# Load the data
id_train_file_path = './Data/train.csv'  ### <-- Update with your data
id_test_file_path = './Data/test.csv'   ### <-- Update with your data
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_aspe_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.aspe['bos_instruct2'], instruct_handler.aspe['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_aspe_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.aspe['bos_instruct2'], instruct_handler.aspe['eos_instruct'])

In [ ]:
# Create T5 utils object
t5_exp = T5Generator(model_checkpoint)

# Tokenize Dataset
id_ds, id_tokenized_ds, ood_ds, ood_tokenized_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Training arguments
training_args = {
    'output_dir':model_out_path,
    'eval_strategy': 'epoch',
    'learning_rate':5e-5,
    'lr_scheduler_type':'cosine',
    'per_device_train_batch_size':8,
    'per_device_eval_batch_size':16,
    'num_train_epochs':30,
    'weight_decay':0.01,
    'warmup_ratio':0.1,
    'save_strategy':'no',
    'load_best_model_at_end':False,
    'push_to_hub':False,
    'eval_accumulation_steps':1,
    'predict_with_generate':True,
    'use_mps_device':use_mps
}

tokenizer_config.json:   0%|          | 0.00/2.57k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/776 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/142 [00:00<?, ?B/s]

Map:   0%|          | 0/1548 [00:00<?, ? examples/s]

Map:   0%|          | 0/381 [00:00<?, ? examples/s]

In [ ]:
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1548
    })
    test: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 381
    })
})

### It'll give us an error if we don't have a validation set (when we try to train). Let's create one now.

In [ ]:
# Split the test dataset in half
train_test_split = id_tokenized_ds['test'].train_test_split(test_size=0.5)
train_test_split

DatasetDict({
    train: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 190
    })
    test: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 191
    })
})

In [ ]:
# Add the "new" 'train' and 'test' splits to the original DatasetDict with their new names
id_tokenized_ds['test'] = train_test_split['train']
id_tokenized_ds['validation'] = train_test_split['test']  # Use 'test' as the validation set
id_tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', '__index_level_0__', 'input_ids', 'attention_mask'],
        num_rows: 1548
    })
    test: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 190
    })
    validation: Dataset({
        features: ['sentenceId', 'raw_text', 'aspectTerms', 'labels', 'text', 'input_ids', 'attention_mask'],
        num_rows: 191
    })
})

### Not mentioned in the video, but run the below cell as well

In [ ]:
!python -m pip install --upgrade pip
!python -m pip install -U accelerate
!python -m pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 24.9 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 35.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 69.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 61.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 64.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 68.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 

In [ ]:
# Train model
model_trainer = t5_exp.train(id_tokenized_ds, **training_args)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Trainer device: cuda:0

Model training started ....


Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Epoch,Training Loss,Validation Loss
1,No log,0.629284
2,No log,0.515485
3,0.800700,0.459596
4,0.800700,0.456378
5,0.800700,0.388768
6,0.432600,0.404750
7,0.432600,0.395033


Epoch,Training Loss,Validation Loss
1,No log,0.629284
2,No log,0.515485
3,0.800700,0.459596
4,0.800700,0.456378
5,0.800700,0.388768
6,0.432600,0.404750
7,0.432600,0.395033
8,0.282500,0.413802
9,0.282500,0.400417
10,0.282500,0.410045


## Inference

In [ ]:
# Load the data
id_train_file_path = './Data/train.csv'
id_test_file_path = './Data/test.csv'
id_tr_df = pd.read_csv(id_train_file_path)
id_te_df = pd.read_csv(id_test_file_path)

# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()

# Set instruction_set1 for InstructABSA-1 and instruction_set2 for InstructABSA-2
instruct_handler.load_instruction_set2()  # same as above, change to instruction_set2

# Set bos_instruct1 for lapt14 and bos_instruct2 for rest14. For other datasets, modify the insructions.py file.
# Same as above. 'joint' changed to 'aspe' in the code below
loader = DatasetLoader(id_tr_df, id_te_df)
if loader.train_df_id is not None:
    loader.train_df_id = loader.create_data_in_aspe_format(loader.train_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.aspe['bos_instruct2'], instruct_handler.aspe['eos_instruct'])
if loader.test_df_id is not None:
    loader.test_df_id = loader.create_data_in_aspe_format(loader.test_df_id, 'term', 'polarity', 'raw_text', 'aspectTerms', instruct_handler.aspe['bos_instruct2'], instruct_handler.aspe['eos_instruct'])

In [ ]:
# Model inference - Loading from Checkpoint
t5_exp = T5Generator(model_out_path)

# Tokenize Datasets
id_ds, id_tokenized_ds, ood_ds, ood_tokenzed_ds = loader.set_data_for_training_semeval(t5_exp.tokenize_function_inputs)

# Get prediction labels - Training set
id_tr_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'train', batch_size = 16)
id_tr_labels = [i.strip() for i in id_ds['train']['labels']]

# Get prediction labels - Testing set
id_te_pred_labels = t5_exp.get_labels(tokenized_dataset = id_tokenized_ds, sample_set = 'test', batch_size = 16)
id_te_labels = [i.strip() for i in id_ds['test']['labels']]

Map:   0%|          | 0/1548 [00:00<?, ? examples/s]

Map:   0%|          | 0/381 [00:00<?, ? examples/s]

Model loaded to:  cuda


100%|██████████| 97/97 [02:47<00:00,  1.72s/it]


Model loaded to:  cuda


100%|██████████| 24/24 [00:47<00:00,  2.00s/it]


In [ ]:
p, r, f1, _ = t5_exp.get_metrics(id_tr_labels, id_tr_pred_labels)
print('Train Precision: ', p)
print('Train Recall: ', r)
print('Train F1: ', f1)

p, r, f1, _ = t5_exp.get_metrics(id_te_labels, id_te_pred_labels)
print('Test Precision: ', p)
print('Test Recall: ', r)
print('Test F1: ', f1)

In [ ]:
from IPython.display import display, Markdown

def absa_inference_single(text, model, tokenizer, bos_instruction, delim_instruction, eos_instruction):
    """
    Thực hiện ABSA inference cho một câu:
      - bos_instruction: prompt đầu
      - delim_instruction: ví dụ " The aspect is "
      - eos_instruction: ví dụ "\noutput:"
    Trả về list các (aspect, polarity).
    """
    # 1. Tạo prompt
    prompt = f"{bos_instruction}{text}{delim_instruction}"

    # 2. Tokenize & generate
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(
        inputs.input_ids.to(model.device),
        max_length=128,
        num_beams=4,
        early_stopping=True,
        use_cache=True
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()

    # 3. Parse kết quả thành list tuples
    pairs = [p.strip() for p in decoded.split(',') if p.strip()]
    results = []
    for p in pairs:
        if ':' in p:
            asp, pol = p.split(':', 1)
            results.append((asp.strip(), pol.strip()))

    # 4. Hiển thị
    display(Markdown(f"**Input:** {text}"))
    display(Markdown(f"**Raw output:** {decoded}"))
    display(Markdown("**Extracted ABSA:**"))
    for asp, pol in results:
        display(Markdown(f"- Aspect: **{asp}**, Polarity: **{pol}**"))

    return results

In [ ]:
instr = InstructionsHandler()
instr.load_instruction_set2()
bos   = instr.aspe['bos_instruct1']
delim = instr.aspe['delim_instruct']
eos   = instr.aspe['eos_instruct']

In [ ]:
# Ví dụ test
sample_text = "I really enjoyed the cinematography but the plot was predictable and dull."
absa_inference_single(sample_text, t5_exp.model, t5_exp.tokenizer, bos, delim, eos)

**Input:** I really enjoyed the cinematography but the plot was predictable and dull.

**Raw output:** cinematography:positive, plot:negative

**Extracted ABSA:**

- Aspect: **cinematography**, Polarity: **positive**

- Aspect: **plot**, Polarity: **negative**

[('cinematography', 'positive'), ('plot', 'negative')]